## Genertaing Check Matrices Based on the 4-fold Cayley complex with PGL/PSL

In [9]:
import numpy as np
import scipy.io
mat13_5= scipy.io.loadmat('ramunujancsscodes_13_5_pgl.mat')
mat17_5= scipy.io.loadmat('ramunujancsscodes_17_5_pgl.mat')
mat29_5= scipy.io.loadmat('ramunujancsscodes_29_5_psl.mat')

mats = [mat13_5, mat29_5, mat17_5] # a family of three in ascending orders of qubits num

# create parity check dictionary to store the parity check matrices for the code family
ZChecks = dict()
XChecks = dict()

In [10]:
def make_check(mat):
    TGVEhor = mat['Expression1'][0] # x check for right action
    TGVEver = mat['Expression1'][1] # x checks for left action
    TGEFhor = mat['Expression1'][2] # z checks for right action
    TGEFver = mat['Expression1'][3] # z checks for left action
    # print(np.linalg.norm(np.matmul(TGVEhor, TGEFhor)- np.matmul(TGVEver, TGEFver)))
    TGVE = np.concatenate([TGVEver, TGVEhor], axis=1) # the concatenated X checks
    TGEF = np.concatenate([TGEFver, TGEFhor], axis=0).transpose() # the concatenated Z checks
    return TGVE, TGEF



#### Check Code Properties

In [11]:
# (1) Check for the CSS codes condition HxHz^T = 0

TGVE, TGEF = make_check(mat13_5)
print(TGEF.shape)
print(TGVE.shape)
T = np.matmul(TGVE, TGEF.transpose())
# np.where(T==1)
# np.where(T>2)
print(len(T[T &2 ==1])) # THe CSS condition is fulfilled.

(23520, 3360)
(120, 3360)
[]


In [21]:
# (2) check the low-density LDPC codes (apated from Lemma 3.2 from Vidick et al 2022)

ma = 1
mb = 1 # current setting We could change that later for better code rate (see later)
Delta = 14 #we are testing for p=13 q =5

Z_row = np.count_nonzero(TGEF, axis=1)
Z_col = np.count_nonzero(TGEF, axis=0)
# print(len(Z_row))
print(f'the maximum weight given Z stabilizer: {np.max(Z_row)}, should be bounded by {4} by Cayley Complexes ')
print(f'the maximum number of Z stabilizers on a given qubit: {np.max(Z_col)}, should be bounded by {ma * mb * Delta}')

X_row = np.count_nonzero(TGVE, axis=1)
X_col = np.count_nonzero(TGVE, axis=0)
print(f'the maximum weight given X stabilizer: {np.max(X_row)}, should be bounded by {2 * Delta}  ')
print(f'the maximum number of X stabilizers on a given qubit: {np.max(X_col)}, should be bounded by {2 * ma * mb}')


the maximum weight given Z stabilizer: 2, should be bounded by 4 by Cayley Complexes 
the maximum number of Z stabilizers on a given qubit: 12, should be bounded by 14
the maximum weight given X stabilizer: 19, should be bounded by 28  
the maximum number of X stabilizers on a given qubit: 1, should be bounded by 2


In [22]:
# 3 Check the code rates adapted from the Lemma 3.3 from Vidick et al 2022 and thm 2.10.

def coderate(TGVE, TGEF):
    X2 = np.shape(TGEF)[0]
    X1 = 2 * np.shape(TGEF)[1]
    X0 = 4 * np.shape(TGVE)[0]
    return (X1 - X2 - X0) / X1

coderate(TGVE, TGEF) # there is no surprise since we choose ma & mb =1 i.e too small.
#TODO: to change ma, mb such that the code rate would be positive. (This, however, comes with storage issue that I am fixing)

-2.5714285714285716

In [8]:
print(np.linalg.matrix_rank(TGVE))
print(np.linalg.matrix_rank(TGEF))

120
3240


In [78]:
for description, mat in zip(['13_5_pgl', '29_5_psl', '17_5_pgl'], mats):
    TGVE, TGEF = make_check(mat)
    XChecks[description] = TGVE
    ZChecks[description] = TGEF



In [79]:
XChecks.keys()

dict_keys(['13_5_pgl', '29_5_psl', '17_5_pgl'])

In [80]:
ZChecks.keys()

dict_keys(['13_5_pgl', '29_5_psl', '17_5_pgl'])